
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Load the dataset with appropriate encoding
file_path = 'data/Corona_NLP_train.csv'
data = pd.read_csv(file_path, encoding='ISO-8859-1')

# Filter out Neutral sentiments
filtered_data = data[data['Sentiment'] != 'Neutral'].copy()

# Map sentiments to binary labels
sentiment_mapping = {
    'Positive': 1,
    'Extremely Positive': 1,
    'Negative': 0,
    'Extremely Negative': 0
}
filtered_data['Sentiment'] = filtered_data['Sentiment'].map(sentiment_mapping)

# Extract the features and labels
X = filtered_data['OriginalTweet'].values
Y = filtered_data['Sentiment'].values

# Split data into fixed proportions: train (60%), test (15%), validation (25%)
train_size = int(0.6 * len(X))
test_size = int(0.15 * len(X))
validation_size = len(X) - train_size - test_size

X_train, X_test, X_valid = (
    X[:train_size], 
    X[train_size:train_size + test_size], 
    X[train_size + test_size:]
)
Y_train, Y_test, Y_valid = (
    Y[:train_size], 
    Y[train_size:train_size + test_size], 
    Y[train_size + test_size:]
)

# Convert data splits into 1-dimensional NumPy arrays
X = np.array(X)
Y = np.array(Y)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_test = np.array(X_test)
Y_test = np.array(Y_test)

X_valid = np.array(X_valid)
Y_valid = np.array(Y_valid)

# Output the shapes of the splits to confirm
print("Data shapes:")
print(f"X: {X.shape}, Y: {Y.shape}")
print(f"X_train: {X_train.shape}, Y_train: {Y_train.shape}")
print(f"X_test: {X_test.shape}, Y_test: {Y_test.shape}")
print(f"X_val: {X_valid.shape}, Y_val: {Y_valid.shape}")


# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.


Data shapes:
X: (33444,), Y: (33444,)
X_train: (20066,), Y_train: (20066,)
X_test: (5016,), Y_test: (5016,)
X_val: (8362,), Y_val: (8362,)


In [5]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

# Part 2: Train a machine learning model pipeline
model = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression(max_iter=1000, solver='lbfgs'))
])

# Train the model on the training data
model.fit(X_train, Y_train)

# Confirm the model is trained and ready
print("Model training completed.")


Model training completed.


In [6]:
from sklearn.metrics import precision_score, recall_score
# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

# Part 3: Evaluate the model on the test set
Y_pred = model.predict(X_test)

precision_0 = precision_score(Y_test, Y_pred, pos_label=0)
precision_1 = precision_score(Y_test, Y_pred, pos_label=1)
recall_0 = recall_score(Y_test, Y_pred, pos_label=0)
recall_1 = recall_score(Y_test, Y_pred, pos_label=1)

# Output precision and recall for both classes
print("Precision and Recall:")
print(f"Class 0 - Precision: {precision_0}, Recall: {recall_0}")
print(f"Class 1 - Precision: {precision_1}, Recall: {recall_1}")

Precision and Recall:
Class 0 - Precision: 0.8581466842336407, Recall: 0.8477223427331887
Class 1 - Precision: 0.8718510405257394, Recall: 0.8808557727775729


In [7]:
# Part 4: Define the cost function
def cost(model, threshold, X, Y):
    # Get predicted probabilities
    probabilities = model.predict_proba(X)[:, 1]  # Probabilities for class 1
    
    # Apply the threshold to make predictions
    predictions = (probabilities >= threshold).astype(int)
    
    # Calculate costs
    costs = np.where(
        (predictions == 0) & (Y == 1),  # False negatives
        1,
        np.where(
            (predictions == 1) & (Y == 0),  # False positives
            5,
            0  # Correct predictions
        )
    )
    
    # Return the average cost
    return np.mean(costs)


# Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost

In [8]:
# Part 5: Find the optimal threshold
thresholds = np.linspace(0, 1, 101)
costs = [cost(model, t, X_test, Y_test) for t in thresholds]
optimal_threshold = thresholds[np.argmin(costs)]
cost_at_optimal_threshold = min(costs)

# Output the optimal threshold and its cost
print(f"Optimal threshold: {optimal_threshold}")
print(f"Cost at optimal threshold: {cost_at_optimal_threshold}")
# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.


Optimal threshold: 0.8
Cost at optimal threshold: 0.27332535885167464


In [ ]:
# Part 6: Compute the cost on the validation set and confidence interval
cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)

# Compute Hoeffding's inequality-based confidence interval
n_valid = len(Y_valid)
delta = 0.01  # 99% confidence level
bound = np.sqrt(np.log(2 / delta) / (2 * n_valid))

cost_interval_valid = (
    cost_at_optimal_threshold_valid - bound,
    cost_at_optimal_threshold_valid + bound
)

# Output the cost and confidence interval
print(f"Cost at optimal threshold on validation set: {cost_at_optimal_threshold_valid}")
print(f"Confidence interval for the cost: {cost_interval_valid}")


assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

Cost at optimal threshold on validation set: 0.2758909351829706
Confidence interval for the cost: (np.float64(0.2592968711066796), np.float64(0.2924849992592616))


In [10]:
# Part 7: Compute variance and confidence interval using Bennett's inequality
# Calculate the random variable C
probabilities_valid = model.predict_proba(X_valid)[:, 1]
predictions_valid = (probabilities_valid >= optimal_threshold).astype(int)
c_values = np.where(
    (predictions_valid == 0) & (Y_valid == 1),  # False negatives
    1,
    np.where(
        (predictions_valid == 1) & (Y_valid == 0),  # False positives
        5,
        0  # Correct predictions
    )
)

# Compute the empirical variance of C
variance_of_C = np.var(c_values, ddof=1)

# Compute Bennett's inequality-based confidence interval
bound_bennett = np.sqrt((2 * variance_of_C * np.log(1 / delta)) / n_valid)
interval_of_C = (
    cost_at_optimal_threshold_valid - bound_bennett,
    cost_at_optimal_threshold_valid + bound_bennett
)

# Output the variance and confidence interval
print(f"Variance of C: {variance_of_C}")
print(f"Confidence interval using Bennett's inequality: {interval_of_C}")


assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

Variance of C: 0.6614662854362982
Confidence interval using Bennett's inequality: (np.float64(0.24889883885855565), np.float64(0.3028830315073855))
